In [1]:
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


D:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [29]:
items = pd.read_csv("All Electronics.csv")

In [31]:
items.head(1)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999"


In [32]:
items.isnull().sum()

name                0
main_category       0
sub_category        0
image               0
link                0
ratings            95
no_of_ratings      95
discount_price    484
actual_price       70
dtype: int64

In [33]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            9600 non-null   object
 1   main_category   9600 non-null   object
 2   sub_category    9600 non-null   object
 3   image           9600 non-null   object
 4   link            9600 non-null   object
 5   ratings         9505 non-null   object
 6   no_of_ratings   9505 non-null   object
 7   discount_price  9116 non-null   object
 8   actual_price    9530 non-null   object
dtypes: object(9)
memory usage: 675.1+ KB


In [35]:
items.describe()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
count,9600,9600,9600,9600,9600,9505,9505,9116,9530
unique,8800,1,1,9321,9600,39,3455,1608,1068
top,TAGG Verve Sense Smartwatch with 1.70'' Large ...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/61ARj+szfW...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.3,1,₹299,₹999
freq,9,9600,9600,7,1,1310,51,481,1540


In [36]:
items.duplicated().sum()

0

In [37]:
items.shape

(9600, 9)

In [39]:
items['#keywords'] = items['name'].str.lower()


In [40]:
items.head(1)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999","redmi 10 power (power black, 8gb ram, 128gb st..."


In [48]:
items['#keywords'].str.split()

0       [redmi, 10, power, (power, black,, 8gb, ram,, ...
1       [oneplus, nord, ce, 2, lite, 5g, (blue, tide,,...
2       [oneplus, bullets, z2, bluetooth, wireless, in...
3       [samsung, galaxy, m33, 5g, (mystique, green,, ...
4       [oneplus, nord, ce, 2, lite, 5g, (black, dusk,...
                              ...                        
9595    [palay®, bts, backpack, for, boys, kpop, bts, ...
9596    [moca, ipad, air, 5th, generation, case, 2022,...
9597    [zebronics, zeb-sound, bomb, 5, tws, v5.0, blu...
9598    [linqs®, original, nxp, chip, |, waterproof, n...
9599    [pragati, systems®, medium, grain, 24x36, inch...
Name: #keywords, Length: 9600, dtype: object

In [50]:
print(items['main_category'].value_counts())
print(items['sub_category'].value_counts())

tv, audio & cameras    9600
Name: main_category, dtype: int64
All Electronics    9600
Name: sub_category, dtype: int64


In [51]:
items.drop(['main_category', 'sub_category'], axis=1, inplace=True)

In [52]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rimjh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [54]:
items['name'] = items['name'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))


In [55]:
items['name']

0       Redmi 10 Power (Power Black, 8GB RAM, 128GB St...
1       OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...
2       OnePlus Bullets Z2 Bluetooth Wireless Ear Earp...
3       Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...
4       OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...
                              ...                        
9595    PALAY® BTS Backpack Boys Kpop BTS Bangtan Scho...
9596    MOCA iPad Air 5th Generation Case 2022 / iPad ...
9597    ZEBRONICS Zeb-Sound Bomb 5 TWS V5.0 Bluetooth ...
9598    LINQS® Original NXP Chip | Waterproof NFC Tag ...
9599    Pragati Systems® Medium Grain 24x36 Inch 10 Oz...
Name: name, Length: 9600, dtype: object

In [56]:
items.head(1)

,name,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...",https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999","redmi 10 power (power black, 8gb ram, 128gb st..."


In [58]:
items['#keywords']

0       redmi 10 power (power black, 8gb ram, 128gb st...
1       oneplus nord ce 2 lite 5g (blue tide, 6gb ram,...
2       oneplus bullets z2 bluetooth wireless in ear e...
3       samsung galaxy m33 5g (mystique green, 6gb, 12...
4       oneplus nord ce 2 lite 5g (black dusk, 6gb ram...
                              ...                        
9595    palay® bts backpack for boys kpop bts bangtan ...
9596    moca ipad air 5th generation case 2022 / ipad ...
9597    zebronics zeb-sound bomb 5 tws v5.0 bluetooth ...
9598    linqs® original nxp chip | waterproof nfc tag ...
9599    pragati systems® medium grain 24x36 inch 10 oz...
Name: #keywords, Length: 9600, dtype: object

In [63]:
items.head(1)

,name,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...",https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999","redmi 10 power (power black, 8gb ram, 128gb st..."


In [59]:
stemmer = PorterStemmer()

In [60]:
def stemming(text):
    words = []
    for word in text.split(' '):
        words.append(stemmer.stem(word))
    return ' '.join(words)

In [64]:
text =  "running jumped playing"
stemming(text)

'run jump play'

In [65]:
items['#keywords'] = items['#keywords'].apply(stemming)

In [66]:
items['#keywords']

0       redmi 10 power (power black, 8gb ram, 128gb st...
1       oneplu nord ce 2 lite 5g (blue tide, 6gb ram, ...
2       oneplu bullet z2 bluetooth wireless in ear ear...
3       samsung galaxi m33 5g (mystiqu green, 6gb, 128...
4       oneplu nord ce 2 lite 5g (black dusk, 6gb ram,...
                              ...                        
9595    palay® bt backpack for boy kpop bt bangtan sch...
9596    moca ipad air 5th gener case 2022 / ipad air 4...
9597    zebron zeb-sound bomb 5 tw v5.0 bluetooth trul...
9598    linqs® origin nxp chip | waterproof nfc tag st...
9599    pragati systems® medium grain 24x36 inch 10 oz...
Name: #keywords, Length: 9600, dtype: object

In [67]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english', dtype = np.uint8)

In [68]:
cv.fit(items['#keywords'])


CountVectorizer(dtype=<class 'numpy.uint8'>, max_features=5000,
                stop_words='english')

In [71]:
vector = cv.transform(items['#keywords']).toarray()

In [72]:
similarity = cosine_similarity(vector)

In [73]:
del(vector)
items.drop(['#keywords'], axis=1, inplace=True)

In [74]:
similarity.shape

(9600, 9600)

In [75]:
similarity[1]

array([0.27272727, 1.        , 0.07106691, ..., 0.        , 0.        ,
       0.        ])

In [76]:
items['name'].sample(10, random_state=5)

8414    ONUMTZ Newly Launched Smart Watch Charging Cab...
2716    boAt Stone 620 Bluetooth Speaker 12W RMS Stere...
1360    VIHM 7 1 Electronic Cleaner kit, Cleaning Kit ...
1214    Apple Lightning USB Camera Adapter, USB 3.0 OT...
420     boAt Rockerz 335 Bluetooth Ear Neckband Qualco...
9269    ENVOUS Newly Launched Fire Bolt Ring Smartwatc...
1994    DIGITEK (DTR-320 FT) Flexible Gorillapod Tripo...
5828    Lava A1 (Candy Blue), Number Talker, Smart AI ...
7166             Kangaro Stapler DS 45L - Colour May Vary
7544    Sellingal Hard Disk Drive Pouch case 2.5" HDD ...
Name: name, dtype: object

In [80]:
def recommender(product):
    product_index = items[items['name'] == product].index[0]
    similarity_list = list(enumerate(similarity[product_index]))
    top_10_similar_product = sorted(similarity_list, key=lambda x : x[1], reverse = True)[1:11]
    for idx, similary in top_10_similar_product:
        print(idx)
        print(items.loc[idx]['name'])

In [81]:
recommender("Lava A1 (Candy Blue), Number Talker, Smart AI Battery, 4 Days Battery Backup, Military Grade Certified, Keypad Mobile")

3343
Lava A1 2021(Blue Silver), Bluetooth Support, Smart AI Battery, Military Grade Certified,4 Days Battery Backup, Keypad Mobile
9194
Lava A1 2021(Black Gold), Bluetooth Support, Smart AI Battery, Military Grade Certified,4 Days Battery Backup, Keypad Mobile
7687
Lava A1 Josh 21(Blue Silver) -Dual Sim,Call Blink Notification,Military Grade Certified 4 Day Battery Backup, Keypad ...
2504
Lava A3 Power (Aqua Blue),Military Grade Certified, Jumbo Battery 10 Days Backup, Dedicated Music Buttons, Keypad Mob...
5275
Lava A5 (Gold), Military Grade Certified 3 Days Battery Backup, Sound Leakage Resistance, Super Battery Mode, Keypad ...
1950
Lava Gem (Blue Gold), Speaker Amplifier, PMMA 2.5D Glass, Military Grade Certified,1.3 MP Camera, Keypad Mobile
1498
Lava Flip, Blue - Dual Sim Keypad GSM Mobile Unique Design, Notification LED Number Talker
5305
Lava Hero Punch(Charcoal Grey),Stylish Design, 3 Day Battery Backup, FM Recording, Keypad Mobile, Basic Mobile
3022
Lava Gem (Gold+Black), Spea

In [82]:
recommender('Lava Flip, Blue - Dual Sim Keypad GSM Mobile Unique Design, Notification LED Number Talker')

2760
Lava Flip, Red - Dual Sim Keypad Mobile Unique Design, Notification LED Number Talker
3558
IKALL K100 Mobile (1.8 Inch, Dual Sim) (Blue)
6731
Nokia 105 Dual SIM, Keypad Mobile Phone Wireless FM Radio | Blue
5828
Lava A1 (Candy Blue), Number Talker, Smart AI Battery, 4 Days Battery Backup, Military Grade Certified, Keypad Mobile
4362
IKALL K100 1.8 Inch Display Keypad Mobile (Dual Sim, 1000 mAh Battery) (Blue)
8880
Vox V11 Keypad Mobile King Talker, Auto Call Recording & Contact Icon (1.8 Inch, Dual Sim Multimedia) (Red)
1051
Nokia 2660 Flip 4G Volte keypad Phone Dual SIM, Dual Screen, inbuilt MP3 Player & Wireless FM Radio | Blue
2205
Nokia 110 Dual SIM (Blue)
2782
Lava A3 (Dark Blue) - Dual Sim Mobile 1750 mAh Big Battery 32 GB Expandable Storage
7687
Lava A1 Josh 21(Blue Silver) -Dual Sim,Call Blink Notification,Military Grade Certified 4 Day Battery Backup, Keypad ...


In [83]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [85]:
pickle.dump(items, open('data.pkl', 'wb'))